In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d tarkkaanko/amazon

Dataset URL: https://www.kaggle.com/datasets/tarkkaanko/amazon
License(s): CC-BY-NC-SA-4.0
  0% 0.00/582k [00:00<?, ?B/s]
100% 582k/582k [00:00<00:00, 73.3MB/s]


In [4]:
 from zipfile import ZipFile
file_name = "/content/amazon.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [5]:
!ls

amazon_reviews.csv  amazon.zip	kaggle.json  sample_data


In [6]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_csv("amazon_reviews.csv")

In [8]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [9]:
df.tail()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4910,4910,"ZM ""J""",1.0,I bought this Sandisk 16GB Class 10 to use wit...,2013-07-23,503,0,0,0,0,0.0,0.0
4911,4911,Zo,5.0,Used this for extending the capabilities of my...,2013-08-22,473,0,0,0,0,0.0,0.0
4912,4912,Z S Liske,5.0,Great card that is very fast and reliable. It ...,2014-03-31,252,0,0,0,0,0.0,0.0
4913,4913,Z Taylor,5.0,Good amount of space for the stuff I want to d...,2013-09-16,448,0,0,0,0,0.0,0.0
4914,4914,Zza,5.0,I've heard bad things about this 64gb Micro SD...,2014-02-01,310,0,0,0,0,0.0,0.0


In [10]:
df.isnull().sum()

Unnamed: 0              0
reviewerName            1
overall                 0
reviewText              1
reviewTime              0
day_diff                0
helpful_yes             0
helpful_no              0
total_vote              0
score_pos_neg_diff      0
score_average_rating    0
wilson_lower_bound      0
dtype: int64

In [11]:
df[df['reviewTime'].isna()==True]

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound


In [12]:
df.dropna(inplace=True)

In [13]:
df.isnull().sum()

Unnamed: 0              0
reviewerName            0
overall                 0
reviewText              0
reviewTime              0
day_diff                0
helpful_yes             0
helpful_no              0
total_vote              0
score_pos_neg_diff      0
score_average_rating    0
wilson_lower_bound      0
dtype: int64

In [14]:
df['helpful_yes'].unique()

array([   0,    1,    3,   15,    4,    5,    2,   38,  422,    6,    7,
         60, 1952,    8,   53, 1428,   22,  112, 1568,   14,   51,   82,
         45])

In [15]:
len(df)

4913

In [16]:
df['wilson_lower_bound'].unique().sum()

17.401868756912137

In [17]:
df['overall'].unique().sum()

15.0

In [18]:
df['overall'].value_counts()

overall
5.0    3921
4.0     526
1.0     244
3.0     142
2.0      80
Name: count, dtype: int64

In [23]:
#preprocessing, removig non alpha char, lowercasing,stemming
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
corpus = []
stemmer = PorterStemmer()
for i in range(0, df.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', df.iloc[i]['reviewText'])
  review = review.lower().split()
  review = [stemmer.stem(word) for word in review if not word in STOPWORDS]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
cv= CountVectorizer(max_features=1500)
X=cv.fit_transform(corpus).toarray()
y=df['overall'].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [27]:
print(X_train.shape)
print(X_test.shape)

(3930, 1500)
(983, 1500)


In [29]:
print(y_train.shape)
print(y_test.shape)

(3930,)
(983,)


In [30]:
scaler=MinMaxScaler()
X_train_Scaled=scaler.fit_transform(X_train)
X_test_Scaled=scaler.transform(X_test)

In [31]:
model_rf=RandomForestClassifier()
model_rf.fit(X_train_Scaled,y_train)

RandomForestClassifier()

In [33]:
print("test accurcy: ",model_rf.score(X_test_Scaled,y_test))
print("train accurcy: ",model_rf.score(X_train_Scaled,y_train))

test accurcy:  0.8026449643947101
train accurcy:  0.9997455470737914


In [34]:
y_pred=model_rf.predict(X_test_Scaled)

In [35]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [37]:
print(cm)

[[ 11   0   0   0  40]
 [  2   0   0   0  15]
 [  3   0   0   0  17]
 [  2   0   0   1 115]
 [  0   0   0   0 777]]
